In [1]:
#Written using Python 3.9.7
#Author: icebreaker, April 2022

from time import sleep, time
from web3 import Web3
import numpy as np
from scipy.stats import norm
from os.path import exists
from hexbytes import HexBytes
from datetime import datetime
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import warnings

pd.options.display.float_format = '{:.2f}'.format
warnings.filterwarnings("ignore")

#ETHERSCAN_API_KEY = os.getenv("ETHERSCAN_API_KEY")
INFURA_KEY = os.getenv("INFURA_KEY")
#INFURA_KEY = os.getenv("INFURA_KEY")
ETHERSCAN_API_KEY = os.getenv("ETHERSCAN_API_KEY")

ETHERSCAN_API_BASE_URL = "https://api.etherscan.io/api"

w3 = Web3(Web3.HTTPProvider("https://mainnet.infura.io/v3/{}".format(INFURA_KEY)))

In [ ]:
##Scrape all Transfer calls on sETH2 ERC-20 - this forms the basis of df containing all uniswap trades and mint(s)
def findTargetTX(startBlock, blockStep, _targetContract, _targetEventTopicSignature, _fileNamePreFix):
    targetContract = _targetContract
    targetEventTopicSignature = _targetEventTopicSignature
    fileNamePreFix = _fileNamePreFix
    latestBlockNumber = startBlock
    currentBlockNumber = w3.eth.get_block_number()
    filteredTXs = []

    while latestBlockNumber < currentBlockNumber:
        borrowFilter = w3.eth.filter(
            {
                "fromBlock": latestBlockNumber,
                "toBlock": latestBlockNumber + blockStep,
                "address": targetContract,
                "topics": [targetEventTopicSignature],
            }
        )
        txLogs = w3.eth.get_filter_logs(borrowFilter.filter_id)

        if len(txLogs):
            ts = pd.to_datetime((w3.eth.get_block(latestBlockNumber)["timestamp"]), unit='s').to_datetime64()
            print(
                datetime.now().strftime("%H:%M:%S"), 
                "Time: {} Covering Block {}-{}: Found {} {} Transfer events".format(
                    ts,
                    latestBlockNumber,
                    latestBlockNumber + blockStep,
                    len(txLogs),
                    fileNamePreFix
                )
            )
            idx = 0
            # For each borrow log found, scrape the txn involving that log and get gas data
            for log in txLogs:
                txnHash = log["transactionHash"]
                print("Getting TX: ",txnHash, " ", idx, " of", len(txLogs))

                txn = w3.eth.get_transaction(txnHash)
                gasLimit = txn["gas"]
                gasPrice = txn["gasPrice"]
                blockNumber = txn["blockNumber"]

                txnReceipt = w3.eth.get_transaction_receipt(txnHash)
                gasUsed = txnReceipt["gasUsed"]

                block = w3.eth.get_block(blockNumber)
                timestamp = block["timestamp"]

                dataStr = log["data"][2:]
                data = [dataStr[i : i + 64] for i in range(0, len(dataStr), 64)]

                filteredTXs.append(
                    {
                        "txHash": txnHash.hex(),
                        "gasLimit": int(gasLimit),
                        "gasUsed": int(gasUsed),
                        "gasPrice": int(gasPrice),
                        "timestamp": timestamp,
                        "blockNumber": blockNumber,
                        "from": txn["from"],
                        "to": txn["to"],
                        'transferFrom' : log.topics[1][-20:].hex(),
                        'transferTo' : log.topics[2][-20:].hex(),
                        'amountTransferedHex' : data[0],
                        'amountTransferedDecimal' : int(data[0],16)/10e17
                    }
                )
                idx += 1
        latestBlockNumber += blockStep
        # sleep to avoid getting rate limited
        sleep(0.001)
        #np.save(fileNamePreFix + "filteredTransactions", np.array(filteredTXs), allow_pickle=True)

    # Save all the order fill data in a pickle file
    print("Found {} total lendingpool ", fileNamePreFix ," fills".format(len(filteredTXs)))
    return filteredTXs

In [ ]:
sETH2 = '0xFe2e637202056d30016725477c5da089Ab0A043A'
topicSig = '0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef' #transfer events, mint is a special case where sender = 0x0

currentBlockNumber = w3.eth.get_block_number()

#Finding and scraping onchain tx's of interest takes approximately ~2 hours via infura
v2Events = findTargetTX(11726304, 10000, sETH2, topicSig, 'sETH2Transfers')
#v2Events = findTargetTX(12726304, 10000, sETH2, topicSig, 'sETH2Transfers') #debug line

columnsFromScrape = ["txHash","gasLimit","gasUsed","gasPrice", "timestamp","blockNumber","from","to",'transferFrom','transferTo','amountTransferedHex', 'amountTransferedDecimal']
finals = pd.DataFrame(v2Events, columns = columnsFromScrape)
finals['datetime'] = list(map(lambda x: datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S.%f')[:-3], finals["timestamp"]))

combinedFills = pd.concat([finals])
combinedFills.to_csv('transferEvents.csv')

In [ ]:
combinedFills = pd.read_csv("transferEvents.csv", index_col='datetime')
combinedFills['datetime'] = list(map(lambda x: datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S.%f')[:-3], combinedFills["timestamp"]))
combinedFills.index = pd.to_datetime(combinedFills.index) 
type(combinedFills.index)

In [ ]:
##PART TWO, Find natural sell flow from the UniV3 contract, this is defined as:
#Filtering all sETH2 Transfer events where the recevier of the transfer is the UNIv3 sETH2/WETH pool (this captures sETH2 pool inflows)
#Then filtering all TX's where there is no "deposit" event in the logs
#Then removing any transactions where there is also a WETH transfer into the same sETH2/WETH pool (this helps filter out arbs/bots/other non natural flow)

transferTopicSignature = "0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef"
depositTopicSignature = "0xe1fffcc4923d04b559f4d29a8bfc6cda04eb5b0d3c460751c2402c5c5cc9109c"
depositToUniV3Output = '0x000000000000000000000000c36442b4a4522e871399cd717abdd847ab11fe88'
wethAdd = '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'

sellTransactions = []
sellEvents = pd.read_csv("sellEvents.csv")
depositEvents = []

#Download enhanced data from the chain

for log in range(0, len(sellEvents)):
    print("Getting TX ", log, "  of", len(sellEvents))
    txnHash = sellEvents['txHashOfSell'][log]
    #txnHash = '0xb247c4d01e521fa4e5db280ef907b43aa5639e427d5183940a745adc168cd2e8'
    txDetails = w3.eth.get_transaction(txnHash)
    txReceipt = w3.eth.get_transaction_receipt(txnHash)
    txLogLen = len(txReceipt.logs)
    
    depositFound = False
    ethTransferToUniFound = False
    bothTrue = False
    
    for log in range(0,txLogLen):
        #print("Working: " , log/txLogLen, "%")
        if txReceipt.logs[log].address == wethAdd:
            for event in txReceipt.logs[log].topics:

                if event.hex() == depositTopicSignature:
                    ethTransferToUniFound = True
                    print (txnHash, "Found a ETH Depost to Uni event", ethTransferToUniFound)

                if event.hex() == depositToUniV3Output:
                    depositFound = True
                    print(txnHash, "Found a ETH transfer deposit", depositFound)
                    
    if depositFound & ethTransferToUniFound:
        bothTrue = True

    depositEvents.append([txnHash, txDetails.value, depositFound, ethTransferToUniFound, bothTrue])                
    sleep(0.01)
    
depositEvents = pd.DataFrame (depositEvents, columns = ['txnHash', 'value', 'depositFound', 'ethTransferToUniFound', "both"])
depositEvents.to_csv("depositEvents.csv")

In [ ]:
combinedFillsEnhanced = pd.merge(combinedFills, 
                     depositEvents, 
                     left_on ='txHash',
                     right_on ='txnHash',
                     how ='left')
combinedFillsEnhanced.to_csv('combinedFillsEnhanced.csv')

In [3]:
##PART 3 = This is where the analysis starts, if you are running this notebook start here, if you dont want to redownload all the raw data

combinedFillsEnhanced = pd.read_csv("combinedFillsEnhanced.csv", index_col='datetime')
combinedFillsEnhanced['datetime'] = list(map(lambda x: datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S.%f')[:-3], combinedFillsEnhanced["timestamp"]))
combinedFillsEnhanced.index = pd.to_datetime(combinedFillsEnhanced['datetime']) 
combinedFillsEnhanced = combinedFillsEnhanced[combinedFillsEnhanced['datetime'] < '2022-04-26'] #cut to eod 25th april, avoids 26th which is incomplete and makes the DF messy


In [4]:
uniEvents = combinedFillsEnhanced[(combinedFillsEnhanced['transferTo'] == '0x7379e81228514a1d2a6cf7559203998e20598346')] #Only look @ TX's where sETH being sent into the pool
uniEvents = uniEvents[(uniEvents['both'] != True)] #Remove tx's where deposit event occurs in conjunction with 
uniEvents = uniEvents[(uniEvents['depositFound'] != True)]

In [5]:
sellGasUni = uniEvents.gasUsed.mean() #Average gas paid for swapping on univ3
totalSoldOnUni = uniEvents.amountTransferedDecimal.sum() #total Amount sold on Uni
countSellTradesOnUni = uniEvents.amountTransferedDecimal.count() #total Amount sold on Uni
print("All Time Sell Stats:\n Total natural sETH2 sold on uniV3: ",totalSoldOnUni, "\n",
"Average gas used on Uni:", sellGasUni, "\n",
"Count of natural sells on uniV3", countSellTradesOnUni, "\n",
"Average trade size in sETH2", totalSoldOnUni/countSellTradesOnUni)


All Time Sell Stats:
 Total natural sETH2 sold on uniV3:  28853.858815701402 
 Average gas used on Uni: 231468.1588966589 
 Count of natural sells on uniV3 2574 
 Average trade size in sETH2 11.209735359635355


In [6]:
#Collect Mint Events and resample frequncy to day level granularity
mintEvents = combinedFillsEnhanced[combinedFillsEnhanced['transferFrom'] == '0x0000000000000000000000000000000000000000']
mintGas = mintEvents.gasUsed.mean() #Average gas paid for swapping on univ3
totalMinted = mintEvents.amountTransferedDecimal.sum() #total Amount sold on Uni
countOfMintEvents = mintEvents.amountTransferedDecimal.count() #total Amount sold on Uni
print("All Time Mint Stats:\n Total sETH2 minted: ",totalMinted, "\n",
"Average gas used to mint:", mintGas, "\n",
"Times seth2 has been minted", countOfMintEvents, "\n",
"Average mint size in sETH2", totalMinted/countOfMintEvents)



#weeklyRollingMintVolumeAvg = dailyMintVolume.rolling(7).mean()

All Time Mint Stats:
 Total sETH2 minted:  58361.437783318215 
 Average gas used to mint: 99986.31011826544 
 Times seth2 has been minted 6088 
 Average mint size in sETH2 9.586307126037815


In [19]:
frequency = '1d'
dailyUNISellSums = uniEvents.resample(frequency).sum()
dailyUNISellMeans = uniEvents.resample(frequency).mean()

dailyUNISellCounts = uniEvents.resample(frequency).count()
dailyMintSums = mintEvents.resample(frequency).sum()
dailyMintCounts = mintEvents.resample(frequency).count()

In [20]:
consolidatedFacts = pd.DataFrame([], columns=())
consolidatedFacts['avgMintTXGas' + "_" + frequency] = dailyMintSums['gasUsed'] / dailyMintCounts['gasUsed']
consolidatedFacts['sumMintGas'+ "_" + frequency] = dailyMintSums['gasUsed']
consolidatedFacts['mintTXCounts'+ "_" + frequency] = dailyMintCounts['gasUsed']
consolidatedFacts['sumMinted_sETH2'+ "_" + frequency] = dailyMintSums['amountTransferedDecimal']

consolidatedFacts['avgSellTXGas'+ "_" + frequency] = dailyUNISellSums['gasUsed'] / dailyUNISellCounts['gasUsed']
consolidatedFacts['sumSellGas'+ "_" + frequency] = dailyUNISellSums['gasUsed'] 
consolidatedFacts['sellTXCounts'+ "_" + frequency] = dailyUNISellCounts['gasUsed']
consolidatedFacts['sumSold_sETH2'+ "_" + frequency] = dailyUNISellSums['amountTransferedDecimal'] 
consolidatedFacts['avgGasPricePaidOnSell'+ "_" + frequency] = dailyUNISellMeans['gasPrice']


In [29]:
#Calc the end result
gasCostToTransfer_sETH2 = 95000
gasCostToTransfer_ETH = 21000
consolidatedFacts['mintsAvoidable_Full_Volume'] = consolidatedFacts['sumSold_sETH2_1d']  >=  consolidatedFacts['sumMinted_sETH2_1d'] 
consolidatedFacts['mintsPotentiallyCrossableCoverage_Percentage'] = consolidatedFacts['sumSold_sETH2_1d'] /  consolidatedFacts['sumMinted_sETH2_1d'] 
#set max crossable amount at 100% (in the event more sell vol was done than minting)
consolidatedFacts['mintsPotentiallyCrossableCoverage_Percentage'].values[consolidatedFacts['mintsPotentiallyCrossableCoverage_Percentage'].values > 1] = 1
consolidatedFacts['costOfImplementingMaxTheoriticalCross'] = consolidatedFacts['mintsPotentiallyCrossableCoverage_Percentage'] *  consolidatedFacts['sellTXCounts_1d'] * (gasCostToTransfer_ETH + gasCostToTransfer_sETH2)
consolidatedFacts['gasSavingOnMaxTheoriticalCross'] = (consolidatedFacts['mintsPotentiallyCrossableCoverage_Percentage'] *  consolidatedFacts['sumSellGas_1d']) -  consolidatedFacts['costOfImplementingMaxTheoriticalCross']
consolidatedFacts['gasSavingOnMaxTheoriticalCross_PnL_in_ETH_AvgSellGasPx'] = ( consolidatedFacts['avgGasPricePaidOnSell_1d'] / 10e17) * consolidatedFacts['gasSavingOnMaxTheoriticalCross']

In [36]:
consolidatedFacts.to_csv("finalResults.csv")

In [37]:
print("Total Expected Savings, Assuming 100% of everything was crossed:")
consolidatedFacts['gasSavingOnMaxTheoriticalCross_PnL_in_ETH_100GWEI_GasPx'].sum() * 3000

Total Expected Savings, Assuming 100% of everything was crossed:


50083.62789121321